# 场景间数据串联

我们知道不同使用场景下面对的对象客户是不同的,针对的问题也不相同.但数据可不管你这些,我们通常需要保证至少在一个时间尺度中各个场景下的数据是真实的相同的,否则保存这些数据将毫无意义.因此如何在不同场景间串联数据也是我们需要考虑的问题.

这部分不是完全讲的pg,因此算个番外,仅作为一个扩充来丰富整个架构体系.

## 数据串联的形式

通常数据串联的形式可以分为两种:

+ 流式同步,即借助消息队列通过事件同步数据.这种模式的好处是相对实时性更好,但结构更复杂,也更容易鲁棒性不足.

+ 批量同步,即使用定时任务批量的导入数据到另一个数据环境.这种方式自然无法实时更新,一般更新周期会根据需要定为1小时,1天,1周甚至1个月.批量同步通常不会用在业务层因为处理不当可能引起io堵塞影响业务.

## 数据串联的情况分类

通常情况下需要串联的情况可以分为

+ `OLTP`->`HTAP`,即部分业务数据要能尽量实时或者近实时的让管理者维护者监控分析,通常需要同步的是用户信息,业务提供的服务信息等基本信息.

+ `行为事件数据`->`HTAP`,即行为事件要尽可能实时或者近实时的让管理者维护者监控分析,通常需要同步的是用户的交互行为这类数据.

+ `HTAP`->`OLAP`,即将一段时间内的数据归档保存,以提供一个可以在更大时间尺度上进行分析的条件.通常这类数据会用于生成总结性报表,或者用于构造特征训练模型.

+ `特征数据`->`OLTP`/`OLAP`,一般是带有机器学习算法的应用中会有,特征是处理过的数据,特征即会需要能够快速构造快速获取然后喂入模型中获得结果,又需要可以充分复用用于构造模型.

下面的部分我们会按数据的串联情况分类来介绍对应的处理思路和工具